In [1]:
from torch.utils import data
from skimage import io
import pandas

class SportLoader(data.Dataset):
    def __init__(self, mode, transform=None):
        self.mode = mode
        self.sport = pandas.read_csv(f"{mode}.csv")
        self.img_name = []
        self.label = []
        self.transform = transform
        
        self.img_name = self.sport['names'].to_list()
        self.label = self.sport['label'].to_list()
        
    def __len__(self):
        return len(self.img_name)
        
    def __getitem__(self, index):
        
        image_path = f"{self.mode}/{self.img_name[index]}"
        self.img = io.imread(image_path)
        self.target = self.label[index]
        
        if self.transform:
            self.img = self.transform(self.img)
        
        return self.img, self.target


In [2]:
from torch.utils.data import DataLoader
from torchvision import transforms

train_transforms = transforms.Compose([
    
    transforms.ToPILImage() ,
    transforms.Resize((96, 96), transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor()
])

train_dataset = SportLoader("train", transform=None)
val_dataset = SportLoader("val", transform=None)
# test_dataset = SportLoader("test")

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [3]:
# import random
# import matplotlib.pyplot as plt


# plt.imshow((train_loader.dataset[random.randint(0, len((train_loader.dataset)))])[0])


In [13]:
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.pool6 = nn.MaxPool2d(2, 2)
        self.conv7 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv8 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv9 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.pool10 = nn.MaxPool2d(2, 2)
        self.conv11 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv12 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv13 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.pool14 = nn.MaxPool2d(2, 2)
        self.ftn1 = nn.Flatten()
        self.fc1 = nn.Linear(25088, 1024)
        self.fc2 = nn.Linear(1024, 10)
        
    def forward(self, input):
        output = F.relu(self.conv1(input))
        output = F.relu(self.conv2(output))
        output = F.relu(self.pool(output))
        output = F.relu(self.conv4(output))
        output = F.relu(self.conv5(output))
        output = F.relu(self.pool6(output))
        output = F.relu(self.conv7(output))
        output = F.relu(self.conv8(output))
        output = F.relu(self.conv9(output))
        output = F.relu(self.pool10(output))
        output = F.relu(self.conv11(output))
        output = F.relu(self.conv12(output))
        output = F.relu(self.conv13(output))
        output = F.relu(self.pool14(output))
#         print(output.size())
        output = output.view(output.size(0), -1)
#         output = self.ftn1(output)
        output = self.fc1(output)
        output = self.fc2(output)
        
        return output

In [14]:
import torch
from torch.autograd import Variable

def fit_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, val_loader):
    training_loss = []
    training_accuracy = []
    validation_loss = []
    validation_accuracy = []

    for epoch in range(num_epochs):
        correct_train = 0
        total_train = 0
#         model.train()
        for i, (x, y) in enumerate(train_loader):
            optimizer.zero_grad()
            x = Variable(x.float().view(input_shape))
#             x, y = x.to('cuda'), y.to('cuda')
            pred = model(x)
            train_loss = loss_func(pred, y)
            train_loss.backward()
            optimizer.step()
            
            predicted = torch.max(pred.data, 1)[1]
            total_train += len(y)
            correct_train += (predicted == y).float().sum()
        train_accuracy = 100 * correct_train / float(total_train)
        training_accuracy.append(train_accuracy)
        training_loss.append(train_loss.data)
        
        correct_val = 0
        total_val = 0
        for i, (x, y) in enumerate(val_loader):
            x = Variable(x.float().view(input_shape))
#             x, y = x.to('cuda'), y.to('cuda')
            pred = model(x)
            val_loss = loss_func(pred, y)
            
            predicted = torch.max(pred.data, 1)[1]
            total_val += len(y)
            correct_val += (predicted == y).float().sum()
        
        val_accuracy = 100 * correct_val / float(total_val)
        validation_accuracy.append(val_accuracy)
        
        validation_loss.append(val_loss.data)
        print('Train Epoch: {}/{} Traing_Loss: {} Traing_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy, val_loss.data, val_accuracy))
    return training_loss, training_accuracy, validation_loss, validation_accuracy
            

In [15]:
model = Network()#.to('cuda')
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
input_shape = (-1,3,224,224)
num_epochs = 10


In [8]:
from torchsummary import summary
summary(model, (3, 96, 96))

ModuleNotFoundError: No module named 'torchsummary'

In [ ]:
number_of_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
number_of_params

In [16]:
training_loss, training_accuracy, validation_loss, validation_accuracy = fit_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, val_loader)

Train Epoch: 1/10 Traing_Loss: 2.2820355892181396 Traing_acc: 10.588235% Val_Loss: 2.303558826446533 Val_accuracy: 11.228070%
Train Epoch: 2/10 Traing_Loss: 2.2439920902252197 Traing_acc: 10.823529% Val_Loss: 2.273444890975952 Val_accuracy: 12.280702%
Train Epoch: 3/10 Traing_Loss: 2.2660796642303467 Traing_acc: 12.470589% Val_Loss: 2.2417891025543213 Val_accuracy: 12.280702%
Train Epoch: 4/10 Traing_Loss: 2.351522445678711 Traing_acc: 12.470589% Val_Loss: 2.2622358798980713 Val_accuracy: 12.280702%
Train Epoch: 5/10 Traing_Loss: 2.199739694595337 Traing_acc: 14.705882% Val_Loss: 2.2761425971984863 Val_accuracy: 11.578947%
Train Epoch: 6/10 Traing_Loss: 2.293084144592285 Traing_acc: 12.235294% Val_Loss: 2.2922165393829346 Val_accuracy: 12.280702%
Train Epoch: 7/10 Traing_Loss: 2.3103654384613037 Traing_acc: 13.647058% Val_Loss: 2.2510457038879395 Val_accuracy: 16.491228%
Train Epoch: 8/10 Traing_Loss: 2.0967235565185547 Traing_acc: 15.647058% Val_Loss: 2.2243213653564453 Val_accuracy: 

In [9]:
import matplotlib.pyplot as plt

# visualization
plt.plot(range(num_epochs), training_loss, 'b-', label='Training_loss')
plt.plot(range(num_epochs), validation_loss, 'g-', label='validation_loss')
plt.title('Training & Validation loss')
plt.xlabel('Number of epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.plot(range(num_epochs), training_accuracy, 'b-', label='Training_accuracy')
plt.plot(range(num_epochs), validation_accuracy, 'g-', label='Validation_accuracy')
plt.title('Training & Validation accuracy')
plt.xlabel('Number of epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: name 'training_loss' is not defined